In [70]:
import re
from datetime import datetime
from pathlib import Path
from typing import Optional
from operator import itemgetter
from itertools import combinations

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from myutils.json import load_json, save_json

## All MCCs

raiffeisen data + my data + bank mccs

#### Raiffeisen

In [71]:
raiffeisen_data = pd.read_csv('/Users/yuralytkin/Development/data/raiffeisen_data.csv',
                              usecols=['amount', 'mcc'], dtype={'mcc': str})

raiffeisen_data.head(2)

,amount,mcc
0,2.884034,5261
1,2.787498,5261


In [72]:
raiffeisen_mccs = set(raiffeisen_data.loc[raiffeisen_data['amount'] > 0, 'mcc'].unique().tolist())

len(raiffeisen_mccs)

243

#### My data

In [83]:
money_movement = pd.read_csv('/Users/yuralytkin/Downloads/movementList-2.csv', sep=';')

money_movement['Дата операции'] = money_movement['Дата операции'].apply(lambda x: datetime.strptime(x, '%d.%m.%y'))
money_movement = money_movement.sort_values(by='Дата операции').reset_index(drop=True)

for key in ['Приход', 'Расход']:
    money_movement[key] = money_movement[key].apply(lambda x: float('.'.join(x.split(','))))
    
mcc_re = re.compile('MCC(\d{4,4})')

def find_mcc(description: str) -> Optional[str]:
    match = mcc_re.search(description)
    
    if match is None:
        return np.nan
    else:
        return match.group(1)
    

money_movement['MCC'] = money_movement['Описание операции'].apply(find_mcc)


def is_cashback(description: str) -> bool:
    description = description.lower()
    
    # one of the following contains a cyrillic `c`
    return ('сashback' in description) or ('cashback' in description)


money_movement['is_cashback'] = money_movement['Описание операции'].apply(is_cashback)

money_movement.head(2)

,Тип счёта,Номер счета,Валюта,Дата операции,Референс проводки,Описание операции,Приход,Расход,Unnamed: 8,MCC,is_cashback
0,Текущий зарплатный счёт,40817810004810191309,RUR,2018-12-13,C071312180005185,должок,400.0,0.0,NaN,NaN,False
1,Текущий зарплатный счёт,40817810004810191309,RUR,2018-12-13,C071312180054312,Перевод денежных средств,0.0,200.0,NaN,NaN,False


In [84]:
money_movement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2664 entries, 0 to 2663
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Тип счёта          2664 non-null   object        
 1   Номер счета        2664 non-null   object        
 2   Валюта             2664 non-null   object        
 3   Дата операции      2664 non-null   datetime64[ns]
 4   Референс проводки  2664 non-null   object        
 5   Описание операции  2664 non-null   object        
 6   Приход             2664 non-null   float64       
 7   Расход             2664 non-null   float64       
 8   Unnamed: 8         0 non-null      float64       
 9   MCC                2373 non-null   object        
 10  is_cashback        2664 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(3), object(6)
memory usage: 210.9+ KB


In [85]:
money_movement = money_movement[(~money_movement['MCC'].isna() & (money_movement['Расход'] > 0))\
                                | money_movement['is_cashback']].reset_index(drop=True)

money_movement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2325 entries, 0 to 2324
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Тип счёта          2325 non-null   object        
 1   Номер счета        2325 non-null   object        
 2   Валюта             2325 non-null   object        
 3   Дата операции      2325 non-null   datetime64[ns]
 4   Референс проводки  2325 non-null   object        
 5   Описание операции  2325 non-null   object        
 6   Приход             2325 non-null   float64       
 7   Расход             2325 non-null   float64       
 8   Unnamed: 8         0 non-null      float64       
 9   MCC                2311 non-null   object        
 10  is_cashback        2325 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(3), object(6)
memory usage: 184.0+ KB


In [86]:
my_data_mccs = set(money_movement.loc[money_movement['Расход'] > 0, 'MCC'].unique().tolist())

len(my_data_mccs)

95

In [87]:
money_movement[['Дата операции', 'Приход', 'Расход', 'MCC', 'is_cashback']]\
    .sort_values(by='Дата операции')\
    .to_csv('../data/money_movement.csv', index=False)

#### Banks

In [23]:
banks = load_json('../data/db/banks.json')

In [24]:
banks_mccs = set([
    mcc
    for bank in banks.values()
    for profit_type in bank['categories']
    for mccs in bank['categories'][profit_type].values()
    for mcc in mccs
] + [
    mcc
    for bank in banks.values()
    for mcc in bank['skipped_mccs']
])


len(banks_mccs)

412

In [25]:
all_mccs = set.union(raiffeisen_mccs, my_data_mccs, banks_mccs)

len(all_mccs)

572

In [26]:
save_json(sorted(all_mccs), '../data/db/all_mccs.json')

## MCC Categories

In [38]:
mcc_categories = {
    'restaurants': ['5811', '5812', '5813'],
    'fastfood': ['5814'],
    'fuel': ['5541', '5542', '5983'],
    'taxi': ['4121'],
    'groceries': ['5411', '5422', '5441', '5451', '5499'],
    'pharmacies': ['5912'],
    'cinema': ['7832'],
    'airlines': ['3000', '3001', '3004', '3005', '3006', '3007',
                 '3008', '3009', '3010', '3011', '3012', '3013',
                 '3015', '3016', '3017', '3018', '3020', '3021',
                 '3022', '3025', '3026', '3028', '3029', '3030',
                 '3032', '3033', '3034', '3035', '3037', '3038',
                 '3039', '3040', '3041', '3042', '3043', '3044',
                 '3047', '3048', '3049', '3050', '3051', '3052',
                 '3056', '3057', '3058', '3061', '3063', '3064',
                 '3066', '3068', '3069', '3072', '3075', '3076',
                 '3077', '3078', '3079', '3082', '3088', '3089',
                 '3098', '3099', '3100', '3102', '3103', '3127',
                 '3129', '3132', '3136', '3144', '3161', '3174',
                 '3180', '3181', '3182', '3183', '3184', '3191',
                 '3196', '3206', '3211', '3217', '3219', '3236',
                 '3240', '3245', '3246', '3247', '3248', '3256',
                 '3260', '3261', '3266', '3294', '3296', '3298',
                 '3299', '4511']
}

In [39]:
skipped = {mcc for bank in banks.values() for mcc in bank['skipped_mccs']}

mcc_categories = {
    key: sorted(set(values) - skipped)
    for key, values in mcc_categories.items()
}

In [40]:
categorized_mccs = {
    mcc
    for bank in banks.values()
    for profit_type in bank['categories']
    for category_mccs in bank['categories'][profit_type].values()
    for mcc in category_mccs
}

other_mccs = set(all_mccs) - skipped - categorized_mccs

len(other_mccs)

160

In [41]:
mcc_categories['other'] = sorted(other_mccs)

In [42]:
for s1, s2 in combinations(mcc_categories.values(), 2):
    assert len(set(s1).intersection(set(s2))) == 0

In [43]:
save_json(mcc_categories, '../data/db/mcc_categories.json')